In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import pickle
import bert
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from pyhanlp import *
from bert import run_classifier
from bert import optimization
from bert import tokenization

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import argparse
import sys

W0804 19:50:35.437467 11296 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\bert\optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [2]:
def pretty_print(result):
    df = pd.DataFrame([result]).T
    df.columns = ["values"]
    return df

In [3]:
def create_tokenizer_from_hub_module(bert_model_hub):
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(bert_model_hub)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
    return bert.tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

In [4]:
def make_features(dataset, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN):
    input_example = dataset.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
    features = bert.run_classifier.convert_examples_to_features(input_example, label_list, MAX_SEQ_LENGTH, tokenizer)
    return features

In [5]:
def create_model(bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, labels, num_labels):
    """Creates a classification model."""

    bert_module = hub.Module(bert_model_hub, trainable=True)
    bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
    bert_outputs = bert_module(inputs=bert_inputs, signature="tokens", as_dict=True)

    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = bert_outputs["pooled_output"]

    hidden_size = output_layer.shape[-1].value

    # Create our own layer to tune for politeness data.
    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):

        # Dropout helps prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)

In [6]:
def model_fn_builder(bert_model_hub, num_labels, learning_rate, num_train_steps, num_warmup_steps):
    """Returns `model_fn` closure for TPUEstimator."""
    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
        # TRAIN and EVAL
        if not is_predicting:

            (loss, predicted_labels, log_probs) = create_model(
            bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            train_op = bert.optimization.create_optimizer(
            loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

            # Calculate evaluation metrics. 
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                f1_score = tf.contrib.metrics.f1_score(label_ids, predicted_labels)
                auc = tf.metrics.auc(label_ids, predicted_labels)
                recall = tf.metrics.recall(label_ids, predicted_labels)
                precision = tf.metrics.precision(label_ids, predicted_labels) 
                true_pos = tf.metrics.true_positives(label_ids, predicted_labels)
                true_neg = tf.metrics.true_negatives(label_ids, predicted_labels)   
                false_pos = tf.metrics.false_positives(label_ids, predicted_labels)  
                false_neg = tf.metrics.false_negatives(label_ids, predicted_labels)
                return {
                    "eval_accuracy": accuracy,
                    "f1_score": f1_score,
                    "auc": auc,
                    "precision": precision,
                    "recall": recall,
                    "true_positives": true_pos,
                    "true_negatives": true_neg,
                    "false_positives": false_pos,
                    "false_negatives": false_neg
                }

            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(
            bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            predictions = {'probabilities': log_probs, 'labels': predicted_labels}
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn

In [7]:
def estimator_builder(bert_model_hub, OUTPUT_DIR, SAVE_SUMMARY_STEPS, SAVE_CHECKPOINTS_STEPS, 
                      label_list, LEARNING_RATE, num_train_steps, num_warmup_steps, BATCH_SIZE):

    # Specify outpit directory and number of checkpoint steps to save
    run_config = tf.estimator.RunConfig(
        model_dir=OUTPUT_DIR,
        save_summary_steps=SAVE_SUMMARY_STEPS,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

    model_fn = model_fn_builder(
        bert_model_hub = bert_model_hub,
        num_labels=len(label_list),
        learning_rate=LEARNING_RATE,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps)


    estimator = tf.estimator.Estimator(
        model_fn=model_fn,
        config=run_config,
        params={"batch_size": BATCH_SIZE})
#         use_tpu=False
#         train_batch_size=BATCH_SIZE,
#         eval_batch_size=8,
#         predict_batch_size=8)
    return estimator, model_fn, run_config

In [8]:
def run_on_dfs(train, test, DATA_COLUMN, LABEL_COLUMN, 
               MAX_SEQ_LENGTH = 128,
               BATCH_SIZE = 32,
               LEARNING_RATE = 2e-5,
               NUM_TRAIN_EPOCHS = 3.0,
               WARMUP_PROPORTION = 0.1,
               SAVE_SUMMARY_STEPS = 100,
               SAVE_CHECKPOINTS_STEPS = 10000,
               bert_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"):

    label_list = train[LABEL_COLUMN].unique().tolist()
    
    tokenizer = create_tokenizer_from_hub_module(bert_model_hub)

    train_features = make_features(train, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)
    test_features = make_features(test, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)

    num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

    estimator, model_fn, run_config = estimator_builder(
                                  bert_model_hub, 
                                  OUTPUT_DIR, 
                                  SAVE_SUMMARY_STEPS, 
                                  SAVE_CHECKPOINTS_STEPS, 
                                  label_list, 
                                  LEARNING_RATE, 
                                  num_train_steps, 
                                  num_warmup_steps, 
                                  BATCH_SIZE)

    train_input_fn = bert.run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=False)

    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

    test_input_fn = run_classifier.input_fn_builder(
        features=test_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False)

    result_dict = estimator.evaluate(input_fn=test_input_fn, steps=None)
    return result_dict, estimator

In [9]:
import random
random.seed(10)

In [10]:
OUTPUT_DIR = 'output'

In [11]:
with open('toutiao_cat_data.txt', 'r', encoding='utf8') as f:
    data = f.read().splitlines()

In [12]:
label = []
title = []
keyword = []

for i in range(len(data)):
    label.append(data[i].split('_!_')[1])
    title.append(data[i].split('_!_')[3])
    keyword.append(data[i].split('_!_')[4])

In [13]:
keyword = pd.DataFrame(keyword)
title = pd.DataFrame(title)
label = pd.DataFrame(label)
trainData = pd.concat( [title , keyword], axis=1 )
trainData = pd.concat( [trainData , label], axis=1 )
trainData.columns = ['title', 'keyword', 'label']
trainData = shuffle(trainData)

trainData = trainData[:2000]
trainData.index = range(len(trainData)) # row index從0開始
trainData = trainData.drop(['keyword'], axis=1)
trainData.head()

,title,label
0,日本凭什么拿下印度高铁订单？,113
1,中国人，应该对种族歧视说“不”,113
2,西藏旅游为何不能随便买骨制纪念品？这里告诉你答案！,112
3,奥迪RSQ8路试谍照，或将于明年3月发布,107
4,李亚鹏晒与王菲十年前旧照 曾一起肩并肩做公益,102


In [14]:
# 對dataset label 做編碼
le = LabelEncoder()
trainData.label = le.fit_transform(trainData.label).reshape(-1,1)

In [15]:
trainData.head()

,title,label
0,日本凭什么拿下印度高铁订单？,11
1,中国人，应该对种族歧视说“不”,11
2,西藏旅游为何不能随便买骨制纪念品？这里告诉你答案！,10
3,奥迪RSQ8路试谍照，或将于明年3月发布,6
4,李亚鹏晒与王菲十年前旧照 曾一起肩并肩做公益,2


In [16]:
# 随机选择20%作为测试集，剩余作为训练集
x_train, x_test, y_train, y_test = train_test_split(trainData.title, trainData.label, test_size=0.2, random_state=0)

In [17]:
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [18]:
train = pd.concat( [x_train , y_train], axis=1 )   # axis=1是X軸，axis=0是y軸
test = pd.concat( [x_test , y_test], axis=1 )

In [19]:
train.head()

,title,label
582,天生一对，美好大结局,0
159,5.9涨停预测,4
1827,LOL版块迎来JAG战队 斗鱼首秀开启倒计时,14
318,上联：南来北往东西客，如何对下联？,1
708,中超第十轮，江苏苏宁1:0战胜大连一方挺进联赛前三，你如何评价本场比赛？,3


In [20]:
test.head()

,title,label
405,北斗导航已经很厉害了，为什么国产手机还在用GPS导航？,8
1190,未来自动驾驶真的会让酒驾和疲劳驾驶成历史吗？,6
1132,华为任正非如何管理低绩效员工？,8
731,Google I/O大会落幕，国内一加6将首批支持Android P,8
1754,你们对高考的独家记忆是什么？,7


In [21]:
# myparam = {
#         "DATA_COLUMN": "title",
#         "LABEL_COLUMN": "label",
#         "LEARNING_RATE": 2e-5,
#         "NUM_TRAIN_EPOCHS":3,
#         "bert_model_hub":"https://tfhub.dev/google/bert_chinese_L-12_H-768_A-12/1"
#     }

In [22]:
# result, estimator = run_on_dfs(train, test, **myparam)

In [23]:
DATA_COLUMN = "title"
LABEL_COLUMN = "label" 
MAX_SEQ_LENGTH = 128
BATCH_SIZE = 32
LEARNING_RATE = 2e-5,
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
SAVE_SUMMARY_STEPS = 100
SAVE_CHECKPOINTS_STEPS = 10000
bert_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

In [24]:
label_list = train[LABEL_COLUMN].unique().tolist()

In [25]:
tokenizer = create_tokenizer_from_hub_module(bert_model_hub)

W0804 19:50:41.916016 11296 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\bert\tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [26]:
train_features = make_features(train, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)
test_features = make_features(test, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)

W0804 19:50:42.038173 11296 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\bert\run_classifier.py:774: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.



In [27]:
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [28]:
estimator, model_fn, run_config = estimator_builder(
                                bert_model_hub, 
                                OUTPUT_DIR, 
                                SAVE_SUMMARY_STEPS, 
                                SAVE_CHECKPOINTS_STEPS, 
                                label_list, 
                                LEARNING_RATE, 
                                num_train_steps, 
                                num_warmup_steps, 
                                BATCH_SIZE)

In [29]:
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [30]:
# early_stopping_hook = tf.estimator.experimental.stop_if_no_decrease_hook(
#             estimator=estimator,
#             metric_name='eval_loss',
#             max_steps_without_decrease=num_train_steps,
#             eval_dir=None,
#             min_steps=0,
#             run_every_secs=None,
#             run_every_steps=500      #FLAGS.save_checkpoints_steps
# )

In [31]:
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

W0804 19:50:42.759582 11296 deprecation.py:323] From D:\Anaconda\lib\site-packages\tensorflow\python\training\training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0804 19:50:45.927265 11296 deprecation.py:506] From <ipython-input-5-845caca54516>:25: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0804 19:50:45.968246 11296 deprecation_wrapper.py:119] From D:\Anaconda\lib\site-packages\bert\optimization.py:27: The name tf.train.get_or_create_global_step is deprecated. Please use tf.compat.v1.train.get_or_create_global_step instead.

W0804 19:50:45.970223 11296 de

InvalidArgumentError: Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Assign requires shapes of both tensors to match. lhs shape= [30522,768] rhs shape= [21128,768]
	 [[node save_2/Assign_10 (defined at <ipython-input-31-abb4a36e3c93>:1) ]]

Errors may have originated from an input operation.
Input Source operations connected to node save_2/Assign_10:
 module/bert/embeddings/word_embeddings/adam_v (defined at D:\Anaconda\lib\site-packages\bert\optimization.py:128)

Original stack trace for 'save_2/Assign_10':
  File "D:\Anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File 

In [ ]:
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

In [ ]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
result_dict = estimator.evaluate(input_fn=test_input_fn, steps=None)

In [ ]:
pretty_print(result)